In [3]:
%%bash
pwd
# ensure that pip install pyspark==3.2.1 i.e. the SPARK version matches the pyspark version, otherwise feast apply fails with errorr "blah blah encryption" 

/home/cdsw/scripts


In [2]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore
import os

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
# feature_store = FeatureStore('/home/cdsw/homecredit_feature_repo')  # Initialize the feature store

# entity_info = pd.DataFrame.from_dict(
#     {
#         # entity's join key -> entity values
#         "sk_id_curr": [107919, 122297, 124718],
#         # "event_timestamp" (reserved key) -> timestamps
#         "event_timestamp": [
#             datetime(2022, 12, 7, 12, 33, 20),
#             datetime(2022, 12, 7, 12, 33, 20),
#             datetime(2022, 12, 7, 12, 33, 20),
#         ],
#     }
# )

## Alternatively


# You can also pass a SQL string to generate the above dataframe.
# This is useful for getting all entities in a timeframe from some data source.

store = FeatureStore(repo_path='/home/cdsw/homecredit_feature_repo')

entity_info = f"""
            SELECT sk_id_curr, current_timestamp as event_timestamp FROM  homecredit_processed_data 
            WHERE event_timestamp < '2024-01-01'    
            """
# training_df = store.get_historical_features(
#     entity_df = entity_info,
#     features=[
#          "homecredit_appl_details_view:target",
#          "homecredit_appl_details_view:flag_own_car",
#          "homecredit_appl_details_view:flag_own_realty",
#     ]
# ).to_df()

feature_service = store.get_feature_service("homecredit_appl_details_Fservice")
training_df=store.get_historical_features(features=feature_service, entity_df = entity_info).to_df()
print("----- Example features -----\n")
print(training_df.head())

/home/cdsw/.local/lib/python3.7/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


----- Example features -----

   sk_id_curr         event_timestamp  index  target  code_gender  \
0      100002 2022-12-06 11:40:11.306      0     1.0            0   
1      100012 2022-12-06 11:40:11.306      9     0.0            0   
2      100016 2022-12-06 11:40:11.306     12     0.0            1   
3      100026 2022-12-06 11:40:11.306     22     0.0            1   
4      100038 2022-12-06 11:40:11.306      4     NaN            0   

   flag_own_car  flag_own_realty  cnt_children  amt_income_total  amt_credit  \
0             0                0             0          202500.0    406597.5   
1             0                0             0          135000.0    405000.0   
2             0                0             0           67500.0     80865.0   
3             0                1             1          450000.0    497520.0   
4             1                1             1          180000.0    625500.0   

   ...  cc_name_contract_status_sent_proposal_var  \
0  ...               

In [4]:
print(training_df.shape)
df=training_df

(356251, 722)


<generator object <genexpr> at 0x7fa02fe8e2d0>


In [7]:
y = df['target']
feats = [f for f in df.columns if f not in ['target','sk_id_curr','sk_id_bureau','sk_id_prev','index', 'event_timestamp']]
X = df[feats]
print("X shape: ", X.shape, "    y shape:", y.shape)


KeyboardInterrupt: 

In [4]:
training_df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356251 entries, 0 to 356250
Columns: 722 entries, sk_id_curr to cc_count
dtypes: datetime64[ns](1), float64(563), int32(158)
memory usage: 1.7 GB


In [8]:
import pandas as pd
df = pd.read_csv('/home/cdsw/04_RAW_TRAINING_DATA/processed_data.csv')
print("Raw shape: ", df.shape)

y = df['target']
feats = [f for f in df.columns if f not in ['target','sk_id_curr','sk_id_bureau','sk_id_prev','index', 'event_timestamp']]
X = df[feats]
print("X shape: ", X.shape, "    y shape:", y.shape)

Raw shape:  (356251, 722)


KeyError: 'target'

In [9]:
training_df.head(10)

,sk_id_curr,event_timestamp,index,target,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,...,cc_name_contract_status_sent_proposal_var,cc_name_contract_status_signed_max,cc_name_contract_status_signed_mean,cc_name_contract_status_signed_sum,cc_name_contract_status_signed_var,cc_name_contract_status_nan_max,cc_name_contract_status_nan_mean,cc_name_contract_status_nan_sum,cc_name_contract_status_nan_var,cc_count
0,100005,2022-12-06 09:36:55.743,1,NaN,0,0,0,0,99000.0,222768.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100007,2022-12-06 09:36:55.743,4,0.0,0,0,0,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100014,2022-12-06 09:36:55.743,10,0.0,1,0,0,1,112500.0,652500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100052,2022-12-06 09:36:55.743,45,0.0,1,0,0,1,90000.0,180000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100070,2022-12-06 09:36:55.743,59,0.0,0,1,0,0,540000.0,1227901.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,100077,2022-12-06 09:36:55.743,65,0.0,0,0,0,0,180000.0,1256400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
6,100078,2022-12-06 09:36:55.743,66,0.0,1,1,0,0,180000.0,1035000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,100103,2022-12-06 09:36:55.743,88,0.0,1,0,0,0,72000.0,450000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,100108,2022-12-06 09:36:55.743,91,0.0,1,0,0,3,171000.0,545040.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,100117,2022-12-06 09:36:55.743,17,NaN,0,1,0,0,90000.0,499221.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
training_df.loc[training_df['sk_id_curr'].isin([100002, 100003, 100004])] # life is good :)

,sk_id_curr,event_timestamp,index,target,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,...,cc_name_contract_status_sent_proposal_var,cc_name_contract_status_signed_max,cc_name_contract_status_signed_mean,cc_name_contract_status_signed_sum,cc_name_contract_status_signed_var,cc_name_contract_status_nan_max,cc_name_contract_status_nan_mean,cc_name_contract_status_nan_sum,cc_name_contract_status_nan_var,cc_count
88622,100002,2022-12-06 09:36:55.743,0,1.0,0,0,0,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306082,100003,2022-12-06 09:36:55.743,1,0.0,1,0,1,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306083,100004,2022-12-06 09:36:55.743,2,0.0,0,1,0,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df.head(10)

,Unnamed: 0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_NAME_CONTRACT_STATUS_Sent proposal_VAR,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,0,100002,1.0,0,0,0,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,100004,0.0,0,1,0,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,100006,0.0,1,0,0,0,135000.0,312682.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,4,100007,0.0,0,0,0,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,5,100008,0.0,0,0,0,0,99000.0,490495.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6,100009,0.0,1,1,0,1,171000.0,1560726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,7,100010,0.0,0,1,0,0,360000.0,1530000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,8,100011,0.0,1,0,0,0,112500.0,1019610.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0
9,9,9,100012,0.0,0,0,0,0,135000.0,405000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
